## Prepare

In [1]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Error: must run as root
Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

You should consider upgrading via the '/home/carlo/anaconda3/envs/data_IC/bin/python -m pip install --upgrade pip' command.
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-2k8aa_4w
  Running command git clone --filter=blob:none -q https://github.com/huggingface/transformers /tmp/pip-req-build-2k8aa_4w
  Resolved https://github.com/huggingface/transformers to commit a03f7514dbd1eef22e2704abc679794b9a6c0064
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 7.6 MB 3.6 MB/s            
     |████████████████████████████████| 770 kB 4.0 MB/s            
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5938675 sha256=0ad86c0036f880a20a59d36083e149fcc454bde469afa7bfda47b6a1fba62c23
  Stored in directory: /tmp/pip-ephem-wheel-cache-mg1tky5f/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49

## Imports

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## Dataset

In [ ]:
language = "Swedish"
language_code = "sv-SE"

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", language_code, split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", language_code, split="test", use_auth_token=True)

print(common_voice)

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

## Feature Extractor

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

## Tokenizer

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language=language, task="transcribe")

## Check first datapoint

In [ ]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

## Wrapper for both

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base", language=language, task="transcribe")

## Explore the data

In [ ]:
print(common_voice["train"][0])

## Change sampling rate

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

print(common_voice["train"][0])

## Create Feature Function

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=4)

## Save the data

In [ ]:
dataset_name = "whisper_swedish"

common_voice.push_to_hub(repo_id=dataset_name, private=True)

In [ ]:
common_voice.save('common_voice', '/content/drive/My Drive/common_voice_dataset')